In [64]:
from pathlib import Path

data_path = Path("data/")
image_path = data_path / "events"
train_dir = image_path / "train"
test_dir = image_path / "test"

train_dir, test_dir
output_dir = 'output'

In [65]:

import torch
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [66]:
if torch.cuda.is_available(): 
    dev = "cuda:0" 
else: 
    dev = "cpu" 
device = torch.device(dev)

In [67]:
import matplotlib.pyplot as plt
import torch

from torch import nn
from torchvision import transforms

from torchvision.models import EfficientNet_V2_S_Weights, efficientnet_v2_s
from torchvision.models import ResNet152_Weights, resnet152
# weights = EfficientNet_V2_S_Weights.DEFAULT
# model = efficientnet_v2_s(weights=weights).to(device)
weights = ResNet152_Weights.DEFAULT
model = resnet152(weights=weights).to(device)


Downloading: "https://download.pytorch.org/models/resnet152-f82ba261.pth" to C:\Users\devas/.cache\torch\hub\checkpoints\resnet152-f82ba261.pth
100%|██████████| 230M/230M [00:16<00:00, 15.0MB/s] 


In [68]:
data_transform = transforms.Compose([
    transforms.Resize((224, 224), antialias=False), 
    transforms.ToTensor(), # 2. Turn image values to between 0 & 1 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], # 3. A mean of [0.485, 0.456, 0.406] (across each colour channel)
                         std=[0.229, 0.224, 0.225]) # 4. A standard deviation of [0.229, 0.224, 0.225] (across each colour channel),
])
auto_transforms = weights.transforms()

In [69]:
from torchvision import datasets
# train_data = datasets.ImageFolder(root=train_dir, # target folder of images
#                                   transform=data_transform, # transforms to perform on data (images)
#                                   target_transform=None) # transforms to perform on labels (if necessary)
# test_data = datasets.ImageFolder(root=test_dir, 
#                                  transform=data_transform)
train_data = datasets.ImageFolder(root=train_dir, # target folder of images
                                  transform=auto_transforms, # transforms to perform on data (images)
                                  target_transform=None) # transforms to perform on labels (if necessary)
test_data = datasets.ImageFolder(root=test_dir, 
                                 transform=auto_transforms)


In [70]:

class_names = train_data.classes
class_names

['Combat',
 'DestroyedBuildings',
 'Fire',
 'Humanitarian Aid and rehabilitation',
 'Military vehicles and weapons']

In [71]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(dataset=train_data, batch_size=32, shuffle=True)
test_dataloader = DataLoader(dataset=test_data, batch_size=32,shuffle=False)

In [73]:
# # Freeze all base layers in the "features" section of the model (the feature extractor) by setting requires_grad=False
# for param in model.features.parameters():
#     param.requires_grad = False

for param in model.parameters():
    param.requires_grad = False

In [74]:
# Get the length of class_names (one output unit for each class)
output_shape = len(class_names)
# torch.autograd.set_detect_anomaly(True)
# Recreate the classifier layer and seed it to the target device

model.classifier = torch.nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280, out_features=output_shape, bias=True),
    # nn.ReLU6(),   
).to(device)

In [75]:
from torch import nn
loss_fn = nn.CrossEntropyLoss() # this is also called "criterion"/"cost function" in some places
# optimizer = torch.optim.RMSprop(params=model.parameters(), lr=0.01)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [76]:
torch.cuda.set_device(0)

In [77]:
next(model.parameters()).is_cuda

True

In [78]:
# Set the random seeds
try:
    from going_modular.going_modular import engine
except:
    # Get the going_modular scripts
    print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !move pytorch-deep-learning\going_modular .
    !rmdir /s /q pytorch-deep-learning
    from going_modular.going_modular import engine

# Start the timer
from timeit import default_timer as timer 
start_time = timer()

# Setup training and save the results
results = engine.train(model=model,
                       train_dataloader=train_dataloader,
                       test_dataloader=test_dataloader,
                       optimizer=optimizer,
                       loss_fn=loss_fn,
                       epochs=10,
                       device=device)

# End the timer and print out how long it took
end_time = timer()
print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")


  0%|          | 0/10 [00:00<?, ?it/s]

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [ ]:
torch.save(model, 'model.tf')
torch.save(model.state_dict(), 'weights.tf')